In [91]:
import os
import psycopg2
from dotenv import load_dotenv
import pandas as pd
import chardet


load_dotenv()

dis_db_password = os.getenv("REMOTE_POSTGRES_DIS_PASSWORD")

conn_params = {
    'host': 'vsisdb.informatik.uni-hamburg.de',
    'dbname': 'dis-2025',
    'user': 'vsisp42',
    'password': dis_db_password
}


In [92]:
def show_existing_tables():
    conn = psycopg2.connect(**conn_params)

    with conn:
        with conn.cursor() as cur:
            cur.execute(
                """
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = %s
                ORDER BY table_name;
                """,
                (conn_params["user"],),
            )
            tables = cur.fetchall()

            for table in tables:
                print("-", table[0])
show_existing_tables()

- article
- city
- country
- dim_date
- dim_geo
- dim_product
- fact_sales
- productcategory
- productfamily
- productgroup
- region
- shop


In [93]:
with open('resources/sales.csv', 'rb') as f:
    result = chardet.detect(f.read(10000))

print(result)

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [94]:
bad_lines = []
with open('resources/sales.csv', encoding='ISO-8859-1') as f:
    for i, line in enumerate(f, start=1):
        if line.count(';') != 4:  # 4 Semikolons = 5 Spalten
            bad_lines.append((i, line.strip()))

for i, l in bad_lines:
    print(f"Fehler in Zeile {i}: {l}")

Fehler in Zeile 35906: 06.04.2019;12.03.2019;Superstore Dresden;AEG Öko Lavatherm 59850 Sensidry;3;2997,00


In [95]:
def load_csv_data():
    return pd.read_csv(
        'resources/sales.csv',
        encoding='ISO-8859-1',
        sep=';',
        decimal=',',
        on_bad_lines='skip'
    )

df = load_csv_data()
print(df.head())


         Date               Shop                           Article  Sold  \
0  01.01.2019  Superstore Berlin  AEG Öko Lavatherm 59850 Sensidry    25   
1  01.01.2019  Superstore Berlin     AEG Öko-Lavamat Öko Plus 1400    25   
2  01.01.2019  Superstore Berlin              Bauknecht TK Care 6B    13   
3  01.01.2019  Superstore Berlin      Bauknecht WA Sensitive 36 DI     2   
4  01.01.2019  Superstore Berlin                       BenQ DE350P    31   

    Revenue  
0  24975.00  
1  14975.00  
2   3639.74  
3    699.80  
4   8369.69  


In [96]:
def create_star_schema_tables(conn):
    with conn.cursor() as cur:
        cur.execute("""
            CREATE TABLE IF NOT EXISTS dim_product (
                product_id SERIAL PRIMARY KEY,
                article_id INT,
                article_name VARCHAR(255),
                product_group_id INT,
                product_group_name VARCHAR(255),
                product_family_id INT,
                product_family_name VARCHAR(255),
                product_category_id INT,
                product_category_name VARCHAR(255)
            );
        """)

        cur.execute("""
            CREATE TABLE IF NOT EXISTS dim_geo (
                geo_id SERIAL PRIMARY KEY,
                shop_id INT,
                shop_name VARCHAR(255),
                city_id INT,
                city_name VARCHAR(255),
                region_id INT,
                region_name VARCHAR(255),
                country_id INT,
                country_name VARCHAR(255)
            );
        """)

        cur.execute("""
            CREATE TABLE IF NOT EXISTS dim_date (
                date DATE PRIMARY KEY,
                day INT,
                month INT,
                quarter INT,
                year INT
            );
        """)

        cur.execute("""
            CREATE TABLE IF NOT EXISTS fact_sales (
                sales_id SERIAL PRIMARY KEY,
                date Date,
                geo_id INT,
                product_id INT,
                quantity INT,
                revenue NUMERIC,

                CONSTRAINT fk_date FOREIGN KEY (date) REFERENCES dim_date(date),
                CONSTRAINT fk_geo FOREIGN KEY (geo_id) REFERENCES dim_geo(geo_id),
                CONSTRAINT fk_product FOREIGN KEY (product_id) REFERENCES dim_product(product_id)
            );
        """)

    conn.commit()
    print("Star-Schema-Tabellen erfolgreich erstellt.")

with psycopg2.connect(**conn_params) as conn:
    create_star_schema_tables(conn)

Star-Schema-Tabellen erfolgreich erstellt.


In [97]:
def populate_dim_product(conn):
    with conn.cursor() as cur:
         cur.execute("""
            INSERT INTO dim_product (
                article_id,
                article_name,
                product_group_id,
                product_group_name,
                product_family_id,
                product_family_name,
                product_category_id,
                product_category_name
            )
            SELECT
                a.ArticleID,
                a.Name,
                pg.ProductGroupID,
                pg.Name,
                pf.ProductFamilyID,
                pf.Name,
                pc.ProductCategoryID,
                pc.Name
            FROM Article a
                INNER JOIN ProductGroup pg ON pg.productgroupid = a.productgroupid
                INNER JOIN vsisp42.productfamily pf on pf.productfamilyid = pg.productfamilyid
                INNER JOIN vsisp42.productcategory pc on pc.productcategoryid = pf.productcategoryid

        """)
    conn.commit()
    print("dim_product erfolgreich befüllt.")

with psycopg2.connect(**conn_params) as conn:
    populate_dim_product(conn)

dim_product erfolgreich befüllt.


In [98]:
def populate_dim_geo(conn):
    with conn.cursor() as cur:
        cur.execute("""
            INSERT INTO dim_geo (
                shop_id,
                shop_name,
                city_id,
                city_name,
                region_id,
                region_name,
                country_id,
                country_name
            )
            SELECT
                s.ShopID,
                s.Name,
                c.CityID,
                c.Name,
                r.RegionID,
                r.Name,
                co.CountryID,
                co.Name
            FROM Shop s
                INNER JOIN City c ON s.CityID = c.CityID
                INNER JOIN Region r ON c.RegionID = r.RegionID
                INNER JOIN Country co ON r.CountryID = co.CountryID
        """)
    conn.commit()
    print("dim_geo erfolgreich befüllt.")

with psycopg2.connect(**conn_params) as conn:
    populate_dim_geo(conn)

dim_geo erfolgreich befüllt.


In [99]:
def transform_data(df):
    df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y', errors='coerce')
    num_invalid_dates = df['Date'].isna().sum()
    print(f'Entries: {len(df)}\t number of invaled dates: {num_invalid_dates}')
    print(df.head())
    # df_sales = df_sales.dropna(subset=['Date'])

df = load_csv_data()
transform_data(df)

Entries: 77311	 number of invaled dates: 0
        Date               Shop                           Article  Sold  \
0 2019-01-01  Superstore Berlin  AEG Öko Lavatherm 59850 Sensidry    25   
1 2019-01-01  Superstore Berlin     AEG Öko-Lavamat Öko Plus 1400    25   
2 2019-01-01  Superstore Berlin              Bauknecht TK Care 6B    13   
3 2019-01-01  Superstore Berlin      Bauknecht WA Sensitive 36 DI     2   
4 2019-01-01  Superstore Berlin                       BenQ DE350P    31   

    Revenue  
0  24975.00  
1  14975.00  
2   3639.74  
3    699.80  
4   8369.69  


In [100]:
def populate_dim_date(conn, df_sales):

    min_date = df_sales['Date'].min()
    max_date = df_sales['Date'].max()

    print(f"Erzeuge Zeitdimension von {min_date.date()} bis {max_date.date()}")

    date_range = pd.date_range(start=min_date, end=max_date)

    df_time = pd.DataFrame({'date': date_range})
    df_time['day'] = df_time['date'].dt.day
    df_time['month'] = df_time['date'].dt.month
    df_time['quarter'] = df_time['date'].dt.quarter
    df_time['year'] = df_time['date'].dt.year

    with conn.cursor() as cur:
        for _, row in df_time.iterrows():
            cur.execute("""
                INSERT INTO dim_date (date, day, month, quarter, year)
                VALUES (%s, %s, %s, %s, %s)
            """, (row['date'], row['day'], row['month'], row['quarter'], row['year']))

    conn.commit()
    print(f"DimTime befüllt mit {len(df_time)} Tagen.")

with psycopg2.connect(**conn_params) as conn:
    populate_dim_date(conn, df)

Erzeuge Zeitdimension von 2019-01-01 bis 2019-05-31
 DimTime befüllt mit 151 Tagen.


In [ ]:
def insert_fact_sales(conn, df_sales):
    with conn.cursor() as cur:
        for _, row in df_sales.iterrows():
            cur.execute("SELECT date FROM dim_date WHERE date = %s", (row['Date'],))
            time_result = cur.fetchone()
            if not time_result:
                continue
            date = time_result[0]

            # Hole geo_id
            cur.execute("SELECT geo_id FROM dim_geo WHERE shop_name = %s", (row['Shop'],))
            geo_result = cur.fetchone()
            if not geo_result:
                continue
            geo_id = geo_result[0]

            # Hole product_id
            cur.execute("SELECT product_id FROM dim_product WHERE article_name = %s", (row['Article'],))
            product_result = cur.fetchone()
            if not product_result:
                continue
            product_id = product_result[0]

            # Insert Fact Record
            cur.execute("""
                INSERT INTO fact_sales (date, geo_id, product_id, quantity, revenue)
                VALUES (%s, %s, %s, %s, %s)
            """, (date, geo_id, product_id, row['Sold'], row['Revenue']))

    conn.commit()
    print("FactSales erfolgreich befüllt.")

with psycopg2.connect(**conn_params) as conn:
    insert_fact_sales(conn, df)
